<a href="https://colab.research.google.com/github/anziltz000/torrent-to-gdrive/blob/main/Torrent_To_Gdrive_anziltz555.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎯 Torrent to Google Drive Downloader

Download `.torrent` or **magnet link** files using `aria2c` right inside **Google Colab**, and upload them directly to your **Google Drive** — all in one smooth workflow!

> 📦 Built with 💡 AI automation — by `anziltz555`  
> ⚙️ Powered by `aria2c`, `tqdm`, and `Google Colab`  
> 🧠 Created using **AI coding assistance**

---

### 📋 How to Use (All Steps Are Required — Nothing Optional)
Follow these steps **in order** for the downloader to work properly:

---

#### ✅ 1. Mount Your Google Drive
- This step lets Colab save downloaded files into your Drive.

#### ✅ 2. Install Required Modules
- Installs `aria2c` for torrent/magnet downloads and `tqdm` for the progress bar.

#### ✅ 3. Choose Your Download Method
- Enter:
  - `1` to **upload a .torrent file**
  - `2` to **paste a magnet link**

#### ✅ 4. Start Downloading
- The script will use `aria2c` to download the torrent or magnet file.
- A **real-time progress bar** will be shown during the download.

#### ✅ 5. Auto Upload to Google Drive
- Once the download is complete, it will be automatically **moved to your Google Drive** folder:
  - `My Drive/TorrentDownloads/Downloads`
- Existing files in the destination **will be overwritten**.

---

### 📎 Notes
- Works with most public `.torrent` files and magnet links.
- Does **not support password-protected private torrents**.
- Each session in Colab is temporary — re-run from the top if disconnected.

---

> 🧠 **Pro Tip**: You can hide code cells using collapse arrows or toggle buttons for a cleaner UI.


# ***Mount Your Google Drive***

In [ ]:
# STEP 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# ***Install Aria2c Module***

In [ ]:
# STEP 2: Install aria2 and tqdm
!apt install -y aria2
!pip install -q tqdm

# **Choose .torrent file (1) or magent link (2)**

In [ ]:
import os
from google.colab import files

download_dir = "/content/Downloads"
os.makedirs(download_dir, exist_ok=True)

mode = input("Enter '1' to upload a .torrent file or '2' to use a magnet link: ").strip()

if mode == "1":
    uploaded = files.upload()
    torrent_file = list(uploaded.keys())[0]
    download_source = torrent_file
    print(f"📁 Uploaded: {torrent_file}")
elif mode == "2":
    magnet_link = input("🔗 Paste your magnet link here: ").strip()
    download_source = magnet_link
    print("✅ Magnet link received.")
else:
    raise ValueError("❌ Invalid input. Enter '1' or '2'.")


# **START DOWNLOADING**

In [ ]:
import subprocess
from tqdm.notebook import tqdm
import re

print("🚀 Starting aria2c download...\n")

process = subprocess.Popen(
    [
        "aria2c",
        f"--dir={download_dir}",
        "--seed-time=0",
        "--summary-interval=1",
        "--console-log-level=warn",
        "--follow-torrent=mem",
        "--max-connection-per-server=5",
        "--split=5",
        download_source
    ],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True,
    bufsize=1
)

progress_bar = tqdm(total=100, desc="📦 Download Progress", ncols=100)
current_percent = 0

for line in process.stdout:
    match = re.search(r"\s(\d{1,3})%", line)
    if match:
        percent = int(match.group(1))
        if percent > current_percent:
            progress_bar.update(percent - current_percent)
            current_percent = percent

process.wait()
progress_bar.n = 100
progress_bar.refresh()
progress_bar.close()

print("✅ Download complete.")


# **Final Move File To Drive**

In [ ]:
import shutil
import os

gdrive_path = '/content/drive/My Drive/TorrentDownloads'
destination_dir = os.path.join(gdrive_path, "Downloads")
os.makedirs(destination_dir, exist_ok=True)

# Move files/folders, overwriting if they already exist
for item in os.listdir(download_dir):
    src_path = os.path.join(download_dir, item)
    dst_path = os.path.join(destination_dir, item)

    if os.path.exists(dst_path):
        if os.path.isdir(dst_path):
            shutil.rmtree(dst_path)  # Remove existing folder
        else:
            os.remove(dst_path)      # Remove existing file

    shutil.move(src_path, dst_path)
    print(f"✅ Overwritten and moved: {item}")

print(f"\n📁 All downloads moved to: {destination_dir}")
